In [15]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import collections
import bellmanford as bf

In [ ]:
data = pd.read_csv('C:/Users/Ceiec04/Desktop/Prediccion_Alertas_Alimentarias/Datasets/full_RASFF_DATA_16092019.csv', sep=';', 
                   header=0, index_col = 0)
data['DATE_CASE'] =pd.to_datetime(data.DATE_CASE, yearfirst=True, format = '%d/%m/%Y')

In [33]:
def most_frequentpath_month(data = data, month = 1):
    mes_mask = data['DATE_CASE'].map(lambda x: x.month) == month
    mes = data[mes_mask]    
    all_routes = []
    for index, row in mes.iterrows():
        origin=row['COUNT_ORIGEN'].split(",")   
        destiny=row['COUNT_DESTIN'].split(",") 
        concern=row['COUNT_CONCERN'].split(",")
        for i in origin:

            if i!=" " and i!= 'INFOSAN' and i!='Commission Services'and i!='unknown origin'and i!='European Food Safety Authority': 
                for j in destiny:
                    if j!=" " and i!=j and j!= 'INFOSAN' and j!='Commission Services'and j!='unknown origin'and j!='European Food Safety Authority': 
                        route = (i,j)
                        all_routes.append(route)

                        for k in concern:
                            if k!=" " and k!= 'INFOSAN' and k!='Commission Services'and k!='unknown origin'and k!='European Food Safety Authority':
                                route = (i,k)
                                all_routes.append(route)   
                    else:
                        for k in concern:
                            if k!=" " and k!= 'INFOSAN' and k!='Commission Services'and k!='unknown origin'and k!='European Food Safety Authority':
                                route = (i,k)
                                all_routes.append(route)
    
    G = nx.DiGraph((x, y, {'weight': v}) for (x, y), v in Counter(all_routes).items())
    g_distance_dict = {(e1, e2): 1 / weight for e1, e2, weight in G.edges(data='weight')}
    nx.set_edge_attributes(G, g_distance_dict, 'distance')
    total = []
    for  node in G.nodes():
        for node1 in G.nodes():
            props = []
            path_length, path_nodes, negative_cycle = bf.bellman_ford(G, source=node, target=node1,
            weight="distance") 
            props.append(node)
            props.append(node1)
            props.append(path_length)
            props.append(path_nodes)
            props.append(negative_cycle)
            total.append(props)      
    all_shortest_paths = pd.DataFrame.from_records(total, columns=["Origin", "Destination", "Weight", "Path", "Ciclo negativo"])
    all_shortest_paths = all_shortest_paths.loc[all_shortest_paths['Origin'] != all_shortest_paths['Destination']]
    all_shortest_paths['Weight'] = 1/all_shortest_paths['Weight']
    all_shortest_paths = all_shortest_paths[['Origin','Destination',"Weight",'Path']]
    all_shortest_paths = all_shortest_paths.sort_values('Weight',ascending = False).round()
    all_shortest_paths_3 = all_shortest_paths[all_shortest_paths['Path'].map(len) == 3]
    all_shortest_paths_3 = all_shortest_paths_3.sort_values('Weight',ascending = False).round()
    return all_shortest_paths,all_shortest_paths_3

In [ ]:
month=12 #Elegir el mes

In [59]:
all_shortest_paths,all_shortest_paths_3 = most_frequentpath_month(data = data, month = month)

In [61]:
all_shortest_paths[:10] #Paths de longitud=2

,Origin,Destination,Weight,Path
3480,Turkey,Germany,184.0,"[Turkey, Germany]"
10439,India,United Kingdom,117.0,"[India, United Kingdom]"
22,China,Italy,117.0,"[China, Italy]"
3865,Spain,Italy,107.0,"[Spain, Italy]"
3,China,Germany,105.0,"[China, Germany]"
3499,Turkey,Italy,98.0,"[Turkey, Italy]"
4027,Italy,Austria,89.0,"[Italy, Austria]"
7140,Iran,Germany,84.0,"[Iran, Germany]"
3498,Turkey,Spain,80.0,"[Turkey, Spain]"
5,China,Netherlands,79.0,"[China, Netherlands]"


In [63]:
all_shortest_paths_3[:10]#Paths de longitud=3

,Origin,Destination,Weight,Path
1,China,Austria,51.0,"[China, Italy, Austria]"
3844,Spain,Austria,49.0,"[Spain, Italy, Austria]"
3478,Turkey,Austria,47.0,"[Turkey, Italy, Austria]"
3846,Spain,Germany,38.0,"[Spain, Italy, Germany]"
10453,India,Italy,38.0,"[India, Spain, Italy]"
14,China,Portugal,37.0,"[China, Italy, Portugal]"
3857,Spain,Portugal,36.0,"[Spain, Italy, Portugal]"
3489,Turkey,Denmark,36.0,"[Turkey, Germany, Denmark]"
3682,United States,Italy,35.0,"[United States, Spain, Italy]"
3491,Turkey,Portugal,35.0,"[Turkey, Italy, Portugal]"
